# Uploading the Datasets to COLAB VM

In [0]:
import io
from os import path
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
from googleapiclient.discovery import build


In [0]:
# Specify the GDrive file-ID and filename to save the file. 
file_id = '1zsUzSIuZXbRkaM7T64EUtvV6OHWgAZKw'
file_name = 'ferplus.csv'

# Authentificate Google Drive SDK.
auth.authenticate_user()

# Load the drive service.
drive_service = build('drive', 'v3')

# If file is not present, download.
if not path.isfile(file_name):

  # Build download request and media downloader.
  request = drive_service.files().get_media(fileId=file_id)
  output_file = io.FileIO(file_name, mode='wb')
  downloader = MediaIoBaseDownload(output_file, request)
  done = False

  # Download chunks.
  while done is False:
      status, done = downloader.next_chunk()
      print('Download %d%%.' % int(status.progress() * 100))

  print('Download Complete!')

In [1]:
# Check if dataset is there.
!ls -lh

total 288M
drwxr-xr-x 1 root root 4.0K Mar 20 10:19 datalab
-rw-r--r-- 1 root root 288M Mar 20 10:19 ferplus.csv


# Data Preprocessing

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

tf.__version__

'1.6.0'

In [3]:
# read csv with pandas
ferplus = pd.read_csv('ferplus.csv')
ferplus.head

FileNotFoundError: File b'ferplus.csv' does not exist

In [3]:
# Extract and cast pixel intensities.
ferplus.pixels = ferplus.pixels.str.split()
ferplus.pixels = ferplus.pixels.map(lambda p: pd.to_numeric(p, downcast='float'))

# Check type of pixel value.
ferplus.pixels[0].dtype

dtype('float32')

In [0]:
# Filter out noface class.
ferplus = ferplus.query('NF == 0')
# Filter out faces where four or more workers were not sure about the emotion.
ferplus = ferplus.query('unknown < 4')

In [0]:
# Class labels in the usual order.
classes = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']

# Get argmax of class distribution to use as label for each image.
labels = ferplus[classes]
maxlabels = labels.idxmax(axis=1).map(labels.columns.get_loc)
ferplus.insert(loc=12, column='maxlabel', value=maxlabels)

In [0]:
# Split train, test and validation set.
train = ferplus.loc[ferplus['Usage'] == 'Training']
valid = ferplus.loc[ferplus['Usage'] == 'PublicTest']
test = ferplus.loc[ferplus['Usage'] == 'PrivateTest']

In [0]:
x_test = np.array(test['pixels'].values.tolist())
x_train = np.array(train['pixels'].values.tolist())
x_valid = np.array(valid['pixels'].values.tolist())

y_test = test[classes].values
y_train = train[classes].values
y_valid = valid[classes].values


In [178]:
tf.argmax(y_test, axis=1)

<tf.Tensor 'ArgMax_5:0' shape=(3482,) dtype=int64>

## Network and Training Parameters

In [0]:
# images are 48x48 pixels
img_size = 48
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

# Classes and class labels
num_classes = 7

# probability to drop a unit (prevent overfitting)
dropout = 0.3

# training parameters
learning_rate = 0.001
max_steps = 5000
# num_epochs = 10
batch_size = 128

## Model Architecture

In [0]:
# Create the neural network
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['images']

        # The data input is a 1-D vector of 2304 features (48*48 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # 4D Input-Tensor: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, img_size, img_size, 1])

        tf.summary.image("img", x)

        # Convolution Layers with 32 filters and a kernel size of 3.
        conv1 = tf.layers.conv2d(x, 32, 3, padding='same', activation=tf.nn.relu)
        conv1 = tf.layers.conv2d(conv1, 32, 3, padding='same', activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Convolution Layers with 64 filters and a kernel size of 3.
        conv2 = tf.layers.conv2d(conv1, 64, 3, padding='same', activation=tf.nn.relu)
        conv2 = tf.layers.conv2d(conv2, 64, 3, padding='same', activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2.
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        
        # Convolution Layer with 128 filters and a kernel size of 3
        conv3 = tf.layers.conv2d(conv2, 128, 3, padding='same', activation=tf.nn.relu)
        conv3 = tf.layers.conv2d(conv3, 128, 3, padding='same', activation=tf.nn.relu)
        conv3 = tf.layers.conv2d(conv3, 128, 3, padding='same', activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2.
        conv3 = tf.layers.max_pooling2d(conv3, 2, 2)

        # Convolution Layers with 256 filters and a kernel size of 3.
        conv4 = tf.layers.conv2d(conv3, 256, 3, padding='same', activation=tf.nn.relu)
        conv4 = tf.layers.conv2d(conv4, 256, 3, padding='same', activation=tf.nn.relu)
        conv4 = tf.layers.conv2d(conv4, 256, 3, padding='same', activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2.
        conv4 = tf.layers.max_pooling2d(conv4, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer.
        fc = tf.contrib.layers.flatten(conv4)

        # Fully connected layer.
        fc = tf.layers.dense(fc, 1024)
        # Apply Dropout (only when is_training is True).
        fc = tf.layers.dropout(fc, rate=dropout, training=is_training)

        # Output layer, class prediction.
        out = tf.layers.dense(fc, n_classes)

    return out

In [0]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    # Because Dropout have different behavior at training and prediction time, we
    # need to create 2 distinct computation graphs that still share the same weights.
    logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)
    
    # Predictions
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode,
          predictions=pred_classes,
          export_outputs={'classes': tf.estimator.export.PredictOutput(pred_classes)}) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        logits=logits_train,
        labels=tf.nn.softmax(
          tf.cast(labels, dtype=tf.float32)
        )
    ))
    
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model using argmax.
    acc_op = tf.metrics.accuracy(labels=tf.argmax(labels, axis=1), predictions=pred_classes)
    
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [199]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn=model_fn, model_dir='model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbebae3aef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training

In [0]:
!rm -Rf model

In [201]:
# Define the input function for training.
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_train}, y=y_train,
    batch_size=batch_size, num_epochs=None, shuffle=True)

# Define the input function for validation during training.
valid_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_valid}, y=y_valid,
    batch_size=batch_size, num_epochs=None, shuffle=False)

# Specify training operations.
train_spec = tf.estimator.TrainSpec(
    input_fn = train_input_fn,
    max_steps = max_steps
)

# Specify evaluation operations.
eval_spec = tf.estimator.EvalSpec(
    input_fn = valid_input_fn,
    throttle_secs=20,
    start_delay_secs=20,
)

# Train the Model and evaluate periodically
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 20 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
duh
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into model/model.ckpt.
INFO:tensorflow:loss = 1.017662, step = 1
INFO:tensorflow:global_step/sec: 9.77565
INFO:tensorflow:loss = 0.34259996, step = 101 (10.237 sec)
INFO:tensorflow:Saving checkpoints for 185 into model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.3536393.
INFO:tensorflow:Calling model_fn.
duh
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-20-14:17:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.

INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-03-20-14:19:20
INFO:tensorflow:Saving dict for global step 1107: accuracy = 0.6944531, global_step = 1107, loss = 0.20398523
INFO:tensorflow:Calling model_fn.
duh
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-1107
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1108 into model/model.ckpt.
INFO:tensorflow:loss = 0.17822644, step = 1108
INFO:tensorflow:global_step/sec: 9.83894
INFO:tensorflow:loss = 0.19862929, step = 1208 (10.172 sec)
INFO:tensorflow:Saving checkpoints for 1292 into model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.20851615.
INFO:tensorflow:Calling model_fn.
duh
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-20-14:19:42
INFO:tensorflow:Graph 

INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-03-20-14:21:57
INFO:tensorflow:Saving dict for global step 2213: accuracy = 0.7391406, global_step = 2213, loss = 0.19437027
INFO:tensorflow:Calling model_fn.
duh
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-2213
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2214 into model/model.ckpt.
INFO:tensorflow:loss = 0.14384262, step = 2214
INFO:tensorflow:global_step/sec: 9.79024
INFO:tensorflow:loss = 0.10439064, step = 2314 (10.219 sec)
INFO:tensorflow:Saving checkpoints for 2397 into model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.13437231.
INFO:tensorflow:Calling model_fn.
duh
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-0

INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-03-20-14:24:33
INFO:tensorflow:Saving dict for global step 3319: accuracy = 0.7386719, global_step = 3319, loss = 0.21307741
INFO:tensorflow:Calling model_fn.
duh
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-3319
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3320 into model/model.ckpt.
INFO:tensorflow:loss = 0.081034645, step = 3320
INFO:tensorflow:global_step/sec: 9.80489
INFO:tensorflow:loss = 0.076784216, step = 3420 (10.202 sec)
INFO:tensorflow:Saving checkpoints for 3503 into model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.08874335.
INFO:tensorflow:Calling model_fn.
duh
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018

INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-03-20-14:27:09
INFO:tensorflow:Saving dict for global step 4422: accuracy = 0.75039065, global_step = 4422, loss = 0.1971636
INFO:tensorflow:Calling model_fn.
duh
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-4422
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4423 into model/model.ckpt.
INFO:tensorflow:loss = 0.07892527, step = 4423
INFO:tensorflow:global_step/sec: 9.81198
INFO:tensorflow:loss = 0.07489807, step = 4523 (10.195 sec)
INFO:tensorflow:Saving checkpoints for 4606 into model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.07826562.
INFO:tensorflow:Calling model_fn.
duh
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-0

## Evaluation

In [202]:
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_test}, y=y_test,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn)

INFO:tensorflow:Calling model_fn.
duh
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-20-14:29:08
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-20-14:29:09
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.75215393, global_step = 5000, loss = 0.20501716


{'accuracy': 0.75215393, 'global_step': 5000, 'loss': 0.20501716}

# Exporting as tf.SavedModel

In [234]:
# Export the model as a SavedModel for production use
feature_spec = {'images': tf.placeholder(dtype=tf.float32, shape=[None, img_size * img_size])}
serving_input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(feature_spec)

savedmodel_path_bytes = model.export_savedmodel(
    export_dir_base='saved_models/ferplus',
    serving_input_receiver_fn=serving_input_fn,
    as_text=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['classes', 'serving_default']
INFO:tensorflow:Restoring parameters from model/model.ckpt-5000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"saved_models/ferplus/temp-b'1521557551'/saved_model.pbtxt"


b'saved_models/ferplus/1521557551'

'saved_models/ferplus/1521557551'

## Download SavedModel .zip-File

In [0]:
import os
import zipfile

model_timestamp = '1521556168'
model_path = savedmodel_path_bytes.decode("utf-8") # TODO: this


def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))


zipf = zipfile.ZipFile(model_timestamp + '.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('saved_models/ferplus/' + model_timestamp, zipf)
zipf.close()
    

In [0]:
from google.colab import files

files.download(model_timestamp + '.zip')